## Problem 1

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.stats import norm

In [ ]:
# Given information
S0 = 151.03
X = 165
r = 0.0425
coupon = 0.0053
b = r - coupon
ivol = 0.2
current_date = datetime(2022,3,13)
exp_date = datetime(2022,4,15)

In [ ]:
# Function to calculate time to maturity using calendar days
def calculate_ttm(curr_date, expire_date):
    days_to_expiration = (expire_date - curr_date).days
    return days_to_expiration / 365

# Function to calculate d1 and d2
def calculate_d1_d2(S0, X, b, T, ivol):
    d1 = (np.log(S0 / X) + (b + ivol**2 / 2) * T) / (ivol * np.sqrt(T))
    d2 = d1 - ivol * np.sqrt(T)
    return d1, d2

T = calculate_ttm(current_date, exp_date)

# Closed form greeks functions for GBSM
def delta_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        delta = np.exp((b-r)*T) * norm.cdf(d1)
    else:
        delta = np.exp((b-r)*T) * (norm.cdf(d1)-1)
    return delta

def gamma_gbsm(S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    gamma = (norm.pdf(d1)*np.exp((b-r)*T)) / (S0 * ivol *np.sqrt(T))
    return gamma

def vega_gbsm(S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    vega = S0 * np.exp((b-r)*T) * norm.pdf(d1) * np.sqrt(T)
    return vega

def theta_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        theta = -(S0*np.exp((b-r)*T)*norm.pdf(d1)*ivol)/(2*np.sqrt(T))\
                -(b-r)*S0*np.exp((b-r)*T)*norm.cdf(d1)\
                -r*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        theta = -(S0*np.exp((b-r)*T)*norm.pdf(d1)*ivol)/(2*np.sqrt(T))\
                +(b-r)*S0*np.exp((b-r)*T)*norm.cdf(-d1)\
                +r*X*np.exp(-r*T)*norm.cdf(-d2)
    return theta

def rho_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        rho = T*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        rho = -T*X*np.exp(-r*T)*norm.cdf(-d2)
    return rho

def carry_rho_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        carry_rho = T*S0*np.exp((b-r)*T)*norm.cdf(d1)
    else:
        carry_rho = -T*S0*np.exp((b-r)*T)*norm.cdf(-d1)
    return carry_rho

def gbsm(option_type, S0, X, T, ivol, r, b):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        call_value = S0 * np.exp((b-r)*T)*norm.cdf(d1) - X * np.exp(-r*T)*norm.cdf(d2)
        return call_value
    else:
        put_value = X * np.exp(-r*T)*norm.cdf(-d2) - S0 * np.exp((b-r)*T)*norm.cdf(-d1)
        return put_value